<a href="https://colab.research.google.com/github/konevenkatesh/Ontology_PM/blob/main/iprok_schedule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install owlready2
!pip install rdflib

In [16]:
# Import necessary libraries
from owlready2 import *
from datetime import date, datetime, timedelta
from rdflib import Graph

In [17]:
set_log_level(9)

In [18]:
# Define the IproK Schedule Ontology IRI and get/create the ontology
schedule_iri = "https://w3id.org/IproK/schedule#"
schedule_iprok = get_ontology(schedule_iri)

* Owlready2 * Creating new ontology schedule <https://w3id.org/IproK/schedule#>.
* Owlready2 * ADD TRIPLE https://w3id.org/IproK/schedule http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Ontology


In [19]:
# Improt base IproK ontology
iprok = get_ontology("iprok.owl").load()
schedule_iprok.imported_ontologies.append(iprok)

* Owlready2 * Reseting property iprok.hasGlobalId: new triples are now available.
* Owlready2 * Reseting property iprok.hasId: new triples are now available.
* Owlready2 * Reseting property iprok.hasName: new triples are now available.
* Owlready2 * Reseting property iprok.hasDescription: new triples are now available.


* Owlready2 * Creating new ontology iprok <iprok.owl#>.
* Owlready2 * ADD TRIPLE iprok.owl http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Ontology
* Owlready2 *     ...loading ontology iprok from iprok.owl...
* Owlready2 *     ...4 properties found: hasGlobalId, hasId, hasName, hasDescription
* Owlready2 * ADD TRIPLE https://w3id.org/IproK/schedule http://www.w3.org/2002/07/owl#imports https://w3id.org/IproK


In [20]:
with schedule_iprok:

    # ==========================================================================
    # Schedule Management Module (Conceptualization based on Module.docx Fig 3)
    # ==========================================================================
    print("Defining Schedule Management Module...")

    # Core Classes
    class Project(iprok.IproKThing):
        """The overarching container for all project-related information."""
        pass

    class WBS_Element(Project):
        """A general class for any component within the project's Work Breakdown Structure."""
        pass

    class WorkPackage(WBS_Element):
        """A significant grouping of related tasks, often a major deliverable or phase."""
        pass

    class Task(WBS_Element):
        """The fundamental unit of work. Can be a summary task or a leaf-level task."""
        pass

    class TaskTime(iprok.IproKThing):
        """A class to reify temporal attributes of a Task."""
        pass

    class TaskDependency(iprok.IproKThing): # Referred to as TaskRel in Module.docx Fig 3
        """Represents logical relationships between tasks."""
        pass

    class Milestone(Task): # Conceptualized as a specialized Task
        """A significant event in the project, often with zero duration."""
        pass
    # Reusing hasId, hasName, hasGlobalId, hasDescription defined earlier
    # Their domains can be extended here or applied generally
    iprok.hasId.domain.append(Task)
    iprok.hasName.domain.append(Task)
    iprok.hasGlobalId.domain.append(Task)
    iprok.hasDescription.domain.append(Task)

    # Schedule Module - Data Properties
    class projectCategory(DataProperty):
        domain = [Project]
        range = [str]

    class projectOwner(DataProperty):
        domain = [Project]
        range = [str] # Or link to an Actor/Organization class

    class projectAddress(DataProperty):
        domain = [Project]
        range = [str]

    class isCriticalTask(DataProperty, FunctionalProperty):
        domain = [Task]
        range = [bool]

    class isMilestone(DataProperty, FunctionalProperty): # If Milestone is not a direct subclass
        domain = [Task]
        range = [bool]

    class taskPriority(DataProperty, FunctionalProperty):
        domain = [Task]
        range = [int]

    class plannedStartDate(DataProperty, FunctionalProperty):
        domain = [TaskTime]
        range = [datetime]

    class plannedFinishDate(DataProperty, FunctionalProperty):
        domain = [TaskTime]
        range = [datetime]

    class actualStartDate(DataProperty, FunctionalProperty):
        domain = [TaskTime]
        range = [datetime]

    class actualFinishDate(DataProperty, FunctionalProperty):
        domain = [TaskTime]
        range = [datetime]

    class scheduleDuration(DataProperty, FunctionalProperty): # Planned duration from schedule
        domain = [TaskTime]
        range = [timedelta]

    class actualDuration(DataProperty, FunctionalProperty):
        domain = [TaskTime]
        range = [timedelta]

    class remainingDuration(DataProperty, FunctionalProperty):
        domain = [TaskTime]
        range = [timedelta]

    class atCompleteDuration(DataProperty, FunctionalProperty):
        domain = [TaskTime]
        range = [timedelta]

    class taskRelationshipType(DataProperty, FunctionalProperty): # e.g., "FS", "SS"
        domain = [TaskDependency]
        range = [str]

    class lagTime(DataProperty, FunctionalProperty):
        domain = [TaskDependency]
        range = [timedelta] # Can be positive (lag) or negative (lead)

    class constraintType(DataProperty, FunctionalProperty):
        domain = [iprok.Constraint]
        range = [str] # e.g., "MustStartOn", "FinishNoLaterThan"

    class constraintDate(DataProperty, FunctionalProperty):
        domain = [iprok.Constraint]
        range = [datetime]

    # Schedule Module - Object Properties
    class hasWorkPackage(ObjectProperty):
        domain = [Project]
        range = [WorkPackage]

    class hasWBS(ObjectProperty): # Links WorkPackage to its WBS_Elements
        domain = [WorkPackage]
        range = [WBS_Element]

    class hasTask(ObjectProperty): # Links WBS_Element to Tasks
        domain = [WBS_Element]
        range = [Task]

    class hasSubTask(ObjectProperty): # For task decomposition
        domain = [Task]
        range = [Task]
        # Consider making it transitive if appropriate, and ensure acyclicity via other means if needed

    class hasTaskTime(ObjectProperty, FunctionalProperty): # A Task has one TaskTime entity
        domain = [Task]
        range = [TaskTime]

    class hasPredecessorDependency(ObjectProperty): # Task links to a Dependency object
        domain = [Task]
        range = [TaskDependency]
        inverse_property_name = "linksToSuccessorTask" # Conceptual inverse name

    class hasSuccessorDependency(ObjectProperty): # Task links to a Dependency object
        domain = [Task]
        range = [TaskDependency]
        inverse_property_name = "linksToPredecessorTask" # Conceptual inverse name

    # Defining the actual linking properties for TaskDependency
    class linksToPredecessorTask(ObjectProperty, FunctionalProperty):
        domain = [TaskDependency]
        range = [Task]
        # inverse_property = hasSuccessorDependency # This creates a direct inverse

    class linksToSuccessorTask(ObjectProperty, FunctionalProperty):
        domain = [TaskDependency]
        range = [Task]
        # inverse_property = hasPredecessorDependency # This creates a direct inverse

    class hasConstraint(ObjectProperty):
        domain = [Task]
        range = [iprok.Constraint]

    class assignedTo(ObjectProperty): # From WorkPackage to Actor
        domain = [WorkPackage, Task] # Task can also be assigned
        range = [iprok.Actor]


Defining Schedule Management Module...


* Owlready2 * ADD TRIPLE https://w3id.org/IproK/schedule#Project http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Class
* Owlready2 * ADD TRIPLE https://w3id.org/IproK/schedule#Project http://www.w3.org/2000/01/rdf-schema#subClassOf https://w3id.org/IproK#IproKThing
* Owlready2 * ADD TRIPLE https://w3id.org/IproK/schedule#WBS_Element http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Class
* Owlready2 * ADD TRIPLE https://w3id.org/IproK/schedule#WBS_Element http://www.w3.org/2000/01/rdf-schema#subClassOf https://w3id.org/IproK/schedule#Project
* Owlready2 * ADD TRIPLE https://w3id.org/IproK/schedule#WorkPackage http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Class
* Owlready2 * ADD TRIPLE https://w3id.org/IproK/schedule#WorkPackage http://www.w3.org/2000/01/rdf-schema#subClassOf https://w3id.org/IproK/schedule#WBS_Element
* Owlready2 * ADD TRIPLE https://w3id.org/IproK/schedule#Task http://www.w3.org/

In [21]:
schedule_iprok.save(file="iprok_schedule.owl", format="rdfxml")

* Owlready2 * Saving ontology schedule to iprok_schedule.owl...


In [22]:
!pip install rdflib


In [23]:
from rdflib import Graph

g= Graph()
g.parse("iprok_schedule.owl", format="xml")
g.serialize(destination="iprok_schedule.ttl", format="ttl")

<Graph identifier=N3d3a5ba7afbf4957a9dd5e7b63e7f7f9 (<class 'rdflib.graph.Graph'>)>

In [24]:
iprok.destroy()
schedule_iprok.destroy()